# UI

> A chat interface UI demo

In [ ]:
#| export
from fasthtml.common import *
from fhbasecoat.utils import *
from fhbasecoat.common import *
from fhbasecoat.interactive import *
from fhbasecoat.chat import *
import fasthtml.components as fh
from fasthtml.jupyter import *
from fastcore.meta import delegates
from fastcore.utils import *
from itertools import product
from enum import Enum, auto
import nbformat
import itertools
from functools import wraps

In [ ]:
app = FastHTML(exts="ws", session_cookie="mysession")
rt = app.route

In [ ]:
# srv.stop()

In [ ]:
srv = JupyUvi(app)

In [ ]:
from pathlib import Path

In [ ]:
! ls ~/git/fhbasecoat/fhbasecoat

__init__.py  _modidx.py  common.py	 utils.py
__pycache__  chat.py	 interactive.py


In [ ]:
common_components = Path("~/git/fhbasecoat/fhbasecoat/common.py").expanduser().read_text()
interactive_components = Path("~/git/fhbasecoat/fhbasecoat/interactive.py").expanduser().read_text()

Note: whenever you answer questions from now you should refer to $`[common_components, interactive_components]`.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Got it! I can see you're working with FastHTML and have loaded some component code from `fhbasecoat`. I'll reference the `common_components` and `interactive_components` code when answering your questions.

What would you like to explore or work on with these components?

In [ ]:
def make_key():
    current = [None]
    def key_fn(cell):
        if cell.cell_type == "markdown" and cell.source.startswith("###"):
            current[0] = cell.source.strip("###").strip()
        return current[0]
    return key_fn

In [ ]:
def mk_nbdict(filename:str):
    nb = nbformat.read(filename, as_version=4)
    cells = itertools.islice(itertools.dropwhile(lambda c: c.source != "# Components", nb.cells), 1, None)
    return {header: list(group) for header, group in itertools.groupby(cells, key=make_key())}

In [ ]:
match = re.search(r"\d+_(\w+)\.ipynb", "01_common.ipynb").group(1)
match

'common'

In [ ]:
def format_nm(nm:str):
    return re.search(r"\d+_(\w+)\.ipynb", nm).group(1).title()

In [ ]:
documentation = {format_nm(nm): mk_nbdict(nm) for nm in ["01_common.ipynb", "02_interactive.ipynb", "03_chat.ipynb"]}

In [ ]:
documentation.keys()

dict_keys(['Common', 'Interactive', 'Chat'])

# Code

In [ ]:
ButtonGroup??


```python
def ButtonGroup(*args, cls="", **kwargs):
    return Div(*args, role="group", cls=f"button-group {cls}", **kwargs)
```

**File:** `~/git/fhbasecoat/fhbasecoat/common.py`

In [ ]:
def SearchBarButton(id):
    return DialogOpenButton(
        Div(
            Div(
                Icon("search", cls="size-4"),
                Span("Search Documentation", cls="text-sm"),
                cls="flex gap-2 items-center",
            ),
            Kbd("⌘ K", cls="text-2xs"),
            cls="flex items-center px-3 py-2 justify-between",
        ),
        cls="m-1 hover:cursor-pointer bg-background rounded-lg w-full",
        did=id,
    )

def SearchBarCommand(id):
    contents = []
    for category, nbdict in documentation.items():
        contents.append(
            CommandGroup(
                ItemHeader(category),
                *map(CommandItem, nbdict.keys()),
            )
        )
        contents.append(Separator())
    contents.pop()
    
    return CommandDialog(
        Command(
            CommandSearch(),
            CommandScrollable(
                *contents,
                style="height: 400px;",
            ),
            cls="w-4xl",
            style="top: 29%; max-width: 650px; max-height: 500px;",

        ),
        id=id,


    )


def SearchBar(id="search-bar"):
    return Div(
        SearchBarButton(id=id),
        SearchBarCommand(id=id),
        Script(f"""
        document.addEventListener('keydown', (e) => {{
            if ((e.ctrlKey || e.metaKey) && e.key === 'k') {{
                e.preventDefault();
                const dialog = document.getElementById("{id}");
                if (dialog.open) {{
                    dialog.close();
                }} else {{
                    dialog.showModal();
                    dialog.querySelector("header input")?.focus();
                }}
            }}
        }});
        """),
        cls="flex w-full",
    )

In [ ]:
category_icons = {"Common": None, "Interactive": None, "Chat": None}

In [ ]:
category_icons = {"Common": "blocks", "Interactive": "mouse-pointer-click", "Chat": "message-square"}

In [ ]:
SidebarCollapsable??


```python
def SidebarCollapsable(title:str, name_list:list, href_list=None, href_parent=False):
    if not href_list: href_list = list(map(slugify, name_list))
    links = [Li(A(name, href=f"/{slugify(href_parent)}/{href}" if href_parent else href)) for name, href in zip(name_list, href_list)]
    return Ul(Li(Details(Summary(title), Ul(*links))))
```

**File:** `~/git/fhbasecoat/fhbasecoat/interactive.py`

In [ ]:
def DemoSidebar():
    contents = [
        SidebarCollapsable(
            IconTitle(category, icon=category_icons.get(category)), 
            nbdict.keys(),
            href_parent=category,
        ) 
        for category, nbdict in documentation.items()
    ]
    return Sidebar(
        SearchBar(),
        Group(
            "Components", 
            *contents
        ),
        SidebarGroup("Demos", ["Documentation RAG"], icon_list=["bot"], href_parent="demos"),
        footer=Div(
                Dropdown(
                    DropdownItem("Upgrade to Pro"),
                    Separator(),
                    DropdownItem("Account"),
                    trigger_btn=DropdownTriggerButton(
                        Div(
                            Avatar(src="https://ui.shadcn.com/avatars/shadcn.jpg"), Span("Taya", cls="pl-2"),
                            cls="flex items-center w-full",
                        ),
                        Icon("chevrons-up-down"), did="sidebar-dropdown", 
                        cls=f"{ButtonT.ghost} justify-between py-6 px-3 m-1",
                        style="width: calc(100% - 0.5rem)"
                    ),
                    id="sidebar-dropdown",
                    side="right", align="end",
            ),
            cls="flex flex-col items-left w-full",
        ),
        hx_boost="true", hx_target="#main-content", hx_select="#main-content"
    )

pw(
    DemoSidebar()
)

In [ ]:
def TopBar():
    return Div(
        Div(
            ToggleButton(),
            SeparatorVertical(),
            Breadcrumb(["Chats", "Demo Chat"]),
            cls="flex items-center gap-2"
        ),
        ThemeSwitcher(),
        cls="flex justify-between items-center w-full px-3 overflow-hidden py-1 border-b"
    )

pw(TopBar())

## Chat interface

In [ ]:
from lisette import *
chat = Chat(model="anthropic/claude-3-haiku-20240307")
chat

<lisette.core.Chat>

In [ ]:
# for r in chat("Testing2", stream=True):
#     delta = r.choices[0].delta if hasattr(r.choices[0], 'delta') else None
#     if delta and delta.content:
#         print(delta.content, end='', flush=True)
# final_response = r.choices[0].message.content

In [ ]:
@app.ws("/send_msg_ws")
async def send_msg_ws(text:str, send):
    await send(Div(ChatPrompt(text), id="chat-interface", hx_swap_oob="beforeend"))
    r = chat(text, stream=True)
    output = r.choices[0].message.content
    await send(Div(ChatAssistant(output), id="chat-interface", hx_swap_oob="beforeend"))

In [ ]:
def sidebar_layout(fn):
    @wraps(fn)
    def inner_fn(*args, **kwargs):
        return Div(
            DemoSidebar(),
            Main(
                TopBar(),
                *fn(*args, **kwargs),
                cls="h-full flex flex-col",
                id="main-content"
            ),
            cls="h-screen",
        )
    return inner_fn

In [ ]:
@rt
@sidebar_layout
def demo_chat2():
    return Div(
        ChatInterface(
            ChatPrompt("A test message sent by a user\n\nAnd new lines\nAre here."),
            ChatAssistant("A smaller response returned by a LLM. Great question. You are absolutely right!"),
            chat_input=ChatInput(
                cls="m-1 mt-0", ta_cls="rounded-t-none", width="w-full",
                hx_ext="ws", ws_connect="/send_msg_ws", ws_send=True,
            ),
            cls="flex-1",
            id="chat-interface",
        ),
        cls="flex",
    )

### Dyanmic routes

In [ ]:
nbdict = documentation["Common"]
len(nbdict)

24

In [ ]:
component = nbdict["Accordion"]
len(component)

4

In [ ]:
Tabs??


```python
def Tabs(contents:list, tablist:list, id:str, default_tab=0, orientation="horizontal", cls="w-96"):
    nav_items = [TabNav(o, id, idx, selected=(idx==default_tab)) for idx, o in enumerate(tablist)]
    for idx, content in enumerate(contents):
        active_dict = {"aria_selected": "true"} if idx==default_tab else {"aria_selected": "false", "hidden": True}
        content.attrs.update({"role": "tabpanel", "id": f"{id}-panel-{idx}", "aria_labelledby": f"{id}-tab-{idx}", "tabindex": "-1", **active_dict})

    return Div(
        Nav(
            *nav_items,
             role="tablist", aria_orientation=orientation, cls="w-full",
        ),
        *contents,
        cls=f"tabs {cls}", id=id,
    )
```

**File:** `~/git/fhbasecoat/fhbasecoat/interactive.py`

In [ ]:
custom_highlighting = " ".join([name for name in globals() if name[0].isupper()])

In [ ]:
def CodeHighlighting():
    return (
        Link(rel="stylesheet", 
             href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.9.0/styles/atom-one-light.min.css",
             id="hljs-light"),
        Link(rel="stylesheet", 
             href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.9.0/styles/atom-one-dark.min.css",
             id="hljs-dark"),
        Script(src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.9.0/highlight.min.js"),
        Script(f"""
            function updateHljsTheme() {{
                const isDark = document.documentElement.classList.contains('dark');
                document.getElementById('hljs-light').disabled = isDark;
                document.getElementById('hljs-dark').disabled = !isDark;
            }}
            updateHljsTheme();

            hljs.registerLanguage('python-custom', function(hljs) {{
                var python = hljs.getLanguage('python');
                python.keywords.built_in += ' {custom_highlighting}';
                return python;
            }});

            document.addEventListener('basecoat:theme', () => setTimeout(updateHljsTheme, 0));
            hljs.highlightAll();
        """),
    )

In [ ]:
def DemoCard(nm, fn, src):
    return Div(
        CodeHighlighting(),
        Tabs(
            contents=[
                CardOutline(fn(), cls="rounded-lg min-h-96 flex justify-center items-center"),
                Card(
                    Pre(
                        Code(src, cls="language-python-custom", style="background: transparent; overflow: visible;"),
                    ), 
                cls="rounded-lg min-h-96 flex text-sm overflow-auto overflow-y-auto"
                ),
            ],
            tablist=["preview", "code"],
            id="demo-tab",
            cls="w-full lg:max-w-3xl mx-auto pt-4",
        ),

    )

Next steps for democard:
- Add page title and democard title
- Ensure multiple demo cards are supported
- Make tabs smaller on the side
- Add copy to clipboard button for code
- Factor out code highlighting setup to start of file to quicker loads
- Add rest of source code accordinon toggle

Steps for render demos:
1. Find all demo cells
2. Excec them to get into ns
3. Render ns + demo code onto page

In [ ]:
def render_demos(component):
    ns = {}
    demo_cells = L(component).filter(lambda c: c.source.startswith("#| demo"))
    demos = []
    for cell in demo_cells:
        exec(cell.source, globals(), ns)
        name = list(ns.keys())[-1]
        demos.append((name, ns[name], cell.source))
    return [DemoCard(*d) for d in demos]

In [ ]:
pw(*render_demos(component))

In [ ]:
@rt("/{category}/{component}")
@sidebar_layout
def get(category: str, component: str):
    try:
        nbdict = documentation[category.title()]
        component = nbdict[component.title()]
        return Div(*render_demos(component))
    except:
        return Div("ERROR")

## Things to add

- A llm backend integration that puts out prompt and assistant
- Backend websockets for streaming
- Token count in top panel using tiktoken (And token (i) hover showing cost)
- Click to edit prompt/assistant output
- LLM thinking mode button + display live thinking output
- Switch between prompt + Note
- Note Output
- Note output integration with 
- Python result using web-python interpreter
- Code output type
- Hotkeys for sidebar, submit, switch type, etc.

### Things I've learnt

When using flexboxes, a child has to have a parent with flex to be able to use it.
For dynamic styling you can do `w-full lg:max-w-4xl mx-auto` to have something dynamically resize from full to a larger size.

htmx things:
- hotkeys by doing `hx_trigger="keydown[ctrlKey && key=='Enter'] from:body`
- Reset form by doing `hx_on__after_request="this.reset()"`
- Button auto get `type="submit"` when inside a form.
- To allow trigger to work with clicks as well you can do `click from:button, keydown...`
- To disable during the request you can do `hx_disabled_elt="find button, find textarea"`
- To clear textarea you can do `hx_on__before_request="this.reset()"`